<a href="https://www.kaggle.com/code/soheybfarohe/distelbert-fine-tuned-f1-sc-1-000-isot-dataset?scriptVersionId=152459086" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



**loading libraries here**

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
!pip install transformers==4.28.0

In [ ]:
from transformers import AdamW, TrainingArguments
from accelerate import Accelerator

In [ ]:
!pip install klib

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk

import numpy as np
from datasets import load_metric
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
# from torch import nn
# import torch.nn.functional as F
import torch
import re
import string
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import klib as kb 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier


In [ ]:
dftrue = pd.read_csv("../input/fake-and-real-news-dataset/True.csv")
dffalse = pd.read_csv("../input/fake-and-real-news-dataset/Fake.csv")

In [ ]:
dftrue.head()

In [ ]:
dftrue.subject.value_counts()

In [ ]:
dftrue['category'] = 1 # true label
dffalse['category'] = 0 # fake lable

In [ ]:
dftrue.head(7)

In [ ]:
dffalse.head(7)

In [ ]:
label_dict={'fake':0,'true':1}

In [ ]:
dftrue.isna().sum() # no missing values 

In [ ]:
dffalse.isna().sum() 

In [ ]:
df_full = pd.concat([dftrue,dffalse])

In [ ]:
df_full.head()

In [ ]:
kb.cat_plot(df_full)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_word_cloud_from_df(df, text_column_name):
    # Extract the text data from the specified column in the dataframe
    text_data = df[text_column_name].tolist()

    # Combine the text data into a single string
    combined_text = ' '.join(text_data)

    # Create a WordCloud object with some customization options
    wordcloud = WordCloud(background_color='white', width=800, height=400).generate(combined_text)

    # Create a figure and axes object
    fig, ax = plt.subplots()
    ax.set_title('Word cloud for ', loc='center')
    # Display the word cloud
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_axis_off()
    # Show the plot
    plt.show()
create_word_cloud_from_df(df_full,text_column_name='text')

In [ ]:
import nltk
from nltk.probability import FreqDist


def plot_most_frequent_words(df, text_column_name, n=10):
    # Extract the text data from the specified column in the dataframe
    text_data = df[text_column_name].tolist()

    # Combine the text data into a single string
    combined_text = ' '.join(text_data)

    # Tokenize the text into words
    words = nltk.tokenize.word_tokenize(combined_text)

    # Create a frequency distribution of the words
    freq_dist = FreqDist(words)

    # Get the most frequent words
    top_words = freq_dist.most_common(n)

    # Create a bar plot of the top N most frequent words with increased spacing
    fig, ax = plt.subplots()
    ax.barh([word[0] for word in top_words], [word[1] for word in top_words], height=0.5)
    ax.set_title(f'Top {n} Most Frequent Words in column: {text_column_name}')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Word')
    plt.tight_layout()
    plt.show()
plot_most_frequent_words(dftrue,text_column_name = 'subject')
plot_most_frequent_words(dffalse,text_column_name = 'subject')



In [ ]:


true_train, true_test = train_test_split(dftrue, test_size=0.3, random_state=42)
false_train, false_test = train_test_split(dffalse, test_size=0.3, random_state=42)


In [ ]:
print(true_train.shape)
print(false_train.shape)
print(true_test.shape)
print(false_test.shape)



**balanced dataset**

In [ ]:



# Concatenate the dataframes vertically
train = pd.concat([true_train, false_train], axis=0).reset_index(drop=True)


test = pd.concat([true_test, false_test], axis=0).reset_index(drop=True)
# Shuffle the concatenated dataframe
train = shuffle(train)
test = shuffle(test)


In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test.category

In [ ]:


# Set the style of the plot
sns.set_style("darkgrid")

# Plot the label balance using countplot
sns.countplot(data=test, x='category')

# Set the axis labels and title
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Label Balance on test")

# Display the plot
plt.show()


In [ ]:


# Assuming you have the label counts stored in the 'label_counts' variable

# Set the style of the plot
sns.set_style("darkgrid")

# Plot the label balance using countplot
sns.countplot(data=train, x='category')

# Set the axis labels and title
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Label Balance on train partion")

# Display the plot
plt.show()


# conclusion : balanced dataset

# preprocessing data

In [ ]:
train.text[14164]

In [ ]:

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove links, remove punctuation,
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', ' ', text)  # Remove text in square brackets
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove links
    text = re.sub(r'<.*?>+', ' ', text)  # Remove HTML tags
    text = re.sub(r'[{}]+'.format(re.escape(string.punctuation)), ' ', text)  # Remove punctuation
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r'[.]', ' ', text)  # Remove period
    return text


In [ ]:
train.text = train.text.map(clean_text)

test.text = test.text.map(clean_text)




In [ ]:
def add_cls_token(df, content_column='content', subject_column='subject', title_column='title', text_column='text', date_column='date'):
    # Add the desired text and separator
    text_sep = "content is"
    title_sep = "title is"
    additional_text = " [SEP], "

    # Concatenate the columns into the 'content' column
    df[content_column] = "[CLS] at " +df[date_column]+additional_text+ df[subject_column] + additional_text+ title_sep + df[title_column] + additional_text+ text_sep + " " + df[text_column]+ additional_text+" ."

    return df


In [ ]:
train.head()

In [ ]:
train2= add_cls_token(train)
test2 = add_cls_token(test)

In [ ]:
train2.text[14164]

In [ ]:
test2.content[6792]

# Transfomers part 

In [ ]:

# tokenizer = AutoTokenizer.from_pretrained('jy46604790/Fake-News-Bert-Detect')
# model = AutoModel.from_pretrained('jy46604790/Fake-News-Bert-Detect', num_labels=2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

path="distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(path)

model = AutoModelForSequenceClassification.from_pretrained(path)


In [ ]:
model

In [ ]:
 model.to('cuda')

In [ ]:
train3 = train2[['content', 'category']]
test3 = test2[['content', 'category']]


In [ ]:
train3.rename(columns={'category': 'labels'}, inplace=True)
test3.rename(columns={'category': 'labels'}, inplace=True)

In [ ]:
test3.head()

In [ ]:
from datasets import Dataset ,DatasetDict
dataset_dict = DatasetDict({
    "train": Dataset.from_dict(train3),
    "test": Dataset.from_dict(test3)
})
dataset_dict

In [ ]:
def tokenize_function(example):
    return tokenizer(example['content'],
                     max_length=128,
                     pad_to_max_length=True,
                     truncation=True,
                     add_special_tokens=False
                    )


In [ ]:
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets

In [ ]:

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [ ]:
import torch.optim as optim
from transformers import TrainingArguments

learning_rate = 0.0005

training_args = TrainingArguments(
    "test_trainer",
    num_train_epochs=2,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    report_to="tensorboard",
    eval_accumulation_steps=1,
    load_best_model_at_end=True,
    disable_tqdm=False,
    optim="adafactor"
)

# # Define the optimizer
# optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



# Accuracy-score 1.000 ,F1-score 1.000

In [ ]:
trainer.train()

In [ ]:
test_dataset = tokenized_datasets["test"]



# Perform the evaluation using trainer.predict
pred_results = trainer.predict(test_dataset)

# Get the predicted labels and true labels
predicted_labels = pred_results.predictions.argmax(axis=1)
true_labels = pred_results.label_ids



# Calculate the F1 score, precision, and recall
classification_rep = classification_report(true_labels, predicted_labels)
print("Classification Report:")
print(classification_rep)


In [ ]:

confusion_mat = confusion_matrix(true_labels, predicted_labels)


In [ ]:
predicted_labels

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(confusion_mat,cmap= "Blues", linecolor = 'black' , linewidth = 0.7 , annot = True, fmt='' , xticklabels = ['Fake','REAL'] , yticklabels = ['Fake','REAL'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
import torch
import pandas as pd

def extract_dense_768_vectors(dataset, model):
    # Set the model to evaluation mode
    model.eval()

    # Create a new column in the dataset to store the extracted vectors
    dataset['dense_768_representation'] = None

    with torch.no_grad():
        # Iterate through each row in the dataset
        for idx, row in dataset.iterrows():
            # Extract input_ids and attention_mask values from the row and convert to tensors
            input_ids = torch.tensor(row['input_ids'], dtype=torch.long).unsqueeze(0).cuda()
            attention_mask = torch.tensor(row['attention_mask'], dtype=torch.long).unsqueeze(0).cuda()

            # Forward pass through the model to get the dense_768 output
            distilbert_output = model.distilbert(input_ids=input_ids, attention_mask=attention_mask)
            hidden_state = distilbert_output.last_hidden_state

            dense_output = model.pre_classifier(hidden_state[:, 0, :])
            dense_output = dense_output.cpu()

            # Store the dense_768 output as a vector in the dataset
            dataset.at[idx, 'dense_768_representation'] = dense_output.detach().numpy().squeeze()

    return dataset


In [ ]:

train_plus_latent_representation= pd.DataFrame(data=tokenized_datasets["train"])
test_plus_latent_representation= pd.DataFrame(data=tokenized_datasets["test"])

In [ ]:
train_plus_latent_representation

In [ ]:
test_plus_latent_representation = extract_dense_768_vectors(test_plus_latent_representation, model)

In [ ]:
test_plus_latent_representation

In [ ]:
train_plus_latent_representation = extract_dense_768_vectors(train_plus_latent_representation, model)

In [ ]:
train_plus_latent_representation['dense_768_representation'][0].shape

In [ ]:
x_train=train_plus_latent_representation['dense_768_representation'].apply(pd.Series).astype(float)
y_train=train_plus_latent_representation['labels']

x_test=test_plus_latent_representation['dense_768_representation'].apply(pd.Series).astype(float)
y_test=test_plus_latent_representation['labels']


In [ ]:
x_test

In [ ]:

# Define the models
logreg_classifier = LogisticRegression()
nb_classifier = GaussianNB()
svm_classifier = SVC(probability=True)
xgboost_classifier = XGBClassifier(n_estimators=500, learning_rate=0.1)

# Fit the models
logreg_classifier.fit(x_train, y_train)
nb_classifier.fit(x_train, y_train)
svm_classifier.fit(x_train, y_train)
xgboost_classifier.fit(x_train, y_train, early_stopping_rounds=5, eval_set=[(x_test, y_test)])

In [ ]:
models = {'Logistic Regression': logreg_classifier,
          'Naive Bayes': nb_classifier,
          'SVM': svm_classifier,
          'XGBoost': xgboost_classifier}

metrics = ['accuracy', 'precision', 'recall', 'f1-score', 'roc_auc']

results = {}
for name, model in models.items():
    # Predict the labels for the validation set
    y_pred = model.predict(x_test)
    
    # Calculate the metrics
    metrics_values = [accuracy_score(y_test, y_pred),
                      precision_score(y_test, y_pred, zero_division=0),
                      recall_score(y_test, y_pred),
                      f1_score(y_test, y_pred),
                      roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])]
    
    # Store the results
    results[name] = dict(zip(metrics, metrics_values))

# Print the results
print("Classification metrics:\n")
print("{:<20} {:<10} {:<10} {:<10} {:<10} {:<10}".format('', *metrics))
for name, result in results.items():
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}".format(name, 
          result['accuracy'], result['precision'], result['recall'], result['f1-score'], result['roc_auc']))

In [ ]:

# Define the models and metrics
models = ['Logistic Regression', 'Naive Bayes', 'SVM', 'XGBoost']
metrics = ['accuracy', 'precision', 'recall', 'f1-score', 'roc_auc']

# Create a figure with subplots for each metric
fig, axs = plt.subplots(nrows=len(metrics), ncols=1, figsize=(8, 24))

# Loop through each metric and create a bar chart for each model
for i, metric in enumerate(metrics):
    axs[i].bar(models, [results[model][metric] for model in models])
    axs[i].set_ylim([0, 1])
    axs[i].set_ylabel(metric)
    axs[i].grid(axis='y')

# Add a title to the figure
fig.suptitle('Classification Metrics by Model', fontsize=16)

# Display the figure
plt.show()

In [ ]:

models = {'Logistic Regression': logreg_classifier,
          'Naive Bayes': nb_classifier,
          'SVM': svm_classifier,
          'XGBoost': xgboost_classifier}

# Create a dictionary to store the confusion matrices
confusion_matrices = {}

for name, model in models.items():
    # Predict the labels for the validation set
    y_pred = model.predict(x_test)
    
    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Store the confusion matrix
    confusion_matrices[name] = cm

# Print the confusion matrices
for name, cm in confusion_matrices.items():
    print("Confusion matrix for", name)
    print(cm)
    print()


In [ ]:

for i, (name, cm) in enumerate(confusion_matrices.items()):
    plt.subplot(2, 2, i+1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
plt.tight_layout()
plt.show()


# using CNN model

In [ ]:

num_rows = 24
num_cols = 32



In [ ]:
num_rows = 24
num_cols = 32
# because we have 768 vector
x_test_matrices = np.array([row.reshape((num_rows, num_cols)) for row in x_test.values])
x_train_matrices =np.array([row.reshape((num_rows, num_cols)) for row in x_train.values])

In [ ]:
x_test_matrices.shape

In [ ]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# Reshape the arrays to have a fourth dimension for color channels
x_train_reshaped = np.reshape(x_train_matrices, (-1, num_rows, num_cols, 1))
x_test_reshaped = np.reshape(x_test_matrices, (-1, num_rows, num_cols, 1))

# Map binary labels to the range [0, 1]
y_train_mapped = y_train.astype(int)
y_test_mapped = y_test.astype(int)
# Split the training data into training and validation sets
x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(x_train_reshaped, y_train_mapped, test_size=0.2, random_state=42)

# Convert the mapped labels to one-hot encoding
y_train_one_hot = to_categorical(y_train_split)
y_valid_one_hot = to_categorical(y_valid_split)
y_test_one_hot = to_categorical(y_test_mapped)
# Define the batch size
batch_size = 32


In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()

model = Sequential()
model.add(Conv2D(256, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(24, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="valid"))
model.add(Conv2D(128, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="valid"))
model.add(Conv2D(128, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="valid"))
model.add(Dense(units=256, activation="relu"))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(units=128, activation="relu"))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units=2, activation="softmax"))

model.summary()

In [ ]:
from tensorflow import keras
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras import callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(x_train_split, y_train_one_hot,
                    epochs=15,
                    batch_size=batch_size,
                    validation_data=(x_valid_split, y_valid_one_hot),
                    callbacks=[early_stopping])

In [ ]:
def plot_history(history):
    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [ ]:
plot_history(history)

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test_one_hot)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:

# Convert one-hot encoding to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)
# predict classes for test set
valid_y_pred = model.predict(x_test_reshaped)
y_pred_classes = np.argmax(valid_y_pred, axis=1)

# create confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_classes)
print(cm)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Create the confusion matrix heatmap
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', ax=ax)

# Set the axis labels
ax.set_xlabel('Predicted Label')
ax.set_ylabel('Original Label')

# Set the title
ax.set_title('Confusion Matrix for test partion predictions')

# Show the plot

plt.show()